In [353]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline



In [374]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

In [466]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

def get_benchmark(indices,start, end):
    indices = indices

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

def ticker_data(tickers, start, end):
    days = (end-start).days

    index = pd.date_range(start, periods = days, freq='D')
    df = pd.DataFrame(index = index,columns = tickers)

    for ticker in tickers:
        df[ticker] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    df = df.dropna()
    return(df)

def portfolio_stats(portfolio, start, end, rf, leverage, margin_int):
    
    weights = np.array(list(portfolio.values()))
    tickers = list(portfolio.keys())
    data = ticker_data(tickers, start, end)
    mean_returns = data.pct_change().mean()
    cov = data.pct_change().cov()
    
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std

    portfolio_stats = {'ret': portfolio_return, 'stdev':portfolio_std, 'sharpe': sharpe_ratio,
                      'levered_ret': levered_return, 'levered_std':levered_std, 'levered_sharpe': levered_sharpe}
    
    return(portfolio_stats)

def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df

def apply_leverage(results_frame, leverage, margin_int):
    results_frame['levered ret']=[0.0]*len(results_frame)
    results_frame['levered stdev']=[0.0]*len(results_frame)
    results_frame['levered sharpe'] = [0.0]*len(results_frame)
    
    for i in range(len(results_frame)):
        results_frame['levered ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
        results_frame['levered stdev'][i] = results_frame['stdev'][i] * leverage
        results_frame['levered sharpe'][i] = results_frame['levered ret'][i]/results_frame['levered stdev'][i]
        
    return(results_frame)


def calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std
    
    return portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe

def simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int):
    results_matrix = np.zeros((len(mean_returns)+6, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe = calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = levered_return
        results_matrix[4,i] = levered_std
        results_matrix[5,i] = levered_sharpe
        
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+6,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe','levered ret','levered stdev','levered sharpe'] + [ticker for ticker in tickers])
        
    return results_df

def match_risk(portfolios, benchmark, start, end, margin_int):
    standard = standard = get_benchmark([benchmark],start, end)
    benchmark_index = standard[standard['benchmark']==benchmark]
    benchmark_risk = float(benchmark_index['stdev'])
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    leverage = float(benchmark_risk / optimal_portfolio['stdev'])
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)


In [433]:
indices = ['SPY', 'IWM', 'VT', 'QQQ']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
standard = get_benchmark(indices,start, end)
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104145,0.199594,0.521784
2,IWM,0.0968185,0.24876,0.389205
3,VT,0.0850438,0.22475,0.378392
4,QQQ,0.157312,0.214189,0.734452


In [399]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
ticker_df = ticker_data(tickers, start, end)
ticker_df.head()

,SPY,GLD,TLT,LQD
2005-07-22,91.241989,42.450001,57.111092,59.299004
2005-07-25,90.983475,42.509998,56.903557,59.293636
2005-07-26,91.094292,42.250000,56.995152,59.304337
2005-07-27,91.426628,42.439999,56.903557,59.229378
2005-07-28,92.002747,42.750000,57.410233,59.454315


In [354]:
portfolio = {'TLT':0.402064, 'SPY':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

portfolio_stats(portfolio, start, end, rf, leverage, margin_int)

{'ret': 0.08964490680532017,
 'stdev': 0.07975261365803406,
 'sharpe': 1.1240372282932647,
 'levered_ret': 0.12446736020798026,
 'levered_std': 0.1196289204870511,
 'levered_sharpe': 1.0404454015068445}

In [426]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
results_frame.head()

,ret,stdev,sharpe,SPY,GLD,TLT,LQD
0,0.072909,0.093520,0.779609,0.029569,0.020454,0.484479,0.465498
1,0.099940,0.118596,0.842694,0.427589,0.413361,0.022252,0.136798
2,0.090626,0.083567,1.084480,0.348190,0.183038,0.280292,0.188480
3,0.084085,0.079027,1.063998,0.199310,0.235154,0.165036,0.400501
4,0.081112,0.080891,1.002726,0.087554,0.211788,0.307070,0.393587


In [401]:
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['sharpe']==max_sharpe]

,ret,stdev,sharpe,SPY,GLD,TLT,LQD
6468,0.089321,0.079429,1.124543,0.287172,0.163747,0.386964,0.162117


In [414]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

df = ticker_data(tickers, start, end)
num_portfolios = 10000
mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()

leveraged_results = simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int)
leveraged_results.head()

,ret,stdev,sharpe,levered ret,levered stdev,levered sharpe,SPY,GLD,TLT,LQD
0,0.092329,0.097367,0.948250,0.128493,0.146051,0.879781,0.120488,0.323986,0.494597,0.060929
1,0.086349,0.088262,0.978326,0.119524,0.132394,0.902794,0.140439,0.357289,0.109425,0.392848
2,0.093409,0.107904,0.865674,0.130114,0.161856,0.803891,0.426877,0.285155,0.018070,0.269898
3,0.096476,0.108936,0.885625,0.134714,0.163404,0.824427,0.433431,0.320651,0.063359,0.182560
4,0.091822,0.083633,1.097915,0.127732,0.125449,1.018201,0.305456,0.238433,0.294742,0.161369


In [418]:
max_lev_sharpe = leveraged_results['levered sharpe'].max()
leveraged_results[leveraged_results['levered sharpe']==max_lev_sharpe]

,ret,stdev,sharpe,levered ret,levered stdev,levered sharpe,SPY,GLD,TLT,LQD
5743,0.090494,0.080496,1.124203,0.125741,0.120744,1.041383,0.296776,0.176891,0.391048,0.135285


In [429]:
leverage = 1.5
margin_int = 0.02
leveraged_portfolios = apply_leverage(results_frame, leverage, margin_int)

In [431]:
leveraged_portfolios['levered stdev'].max()

0.2791913341503632

In [441]:
benchmark = 'SPY'
benchmark_index = standard[standard['benchmark']==benchmark]
benchmark_risk = float(benchmark_index['stdev'])

In [ ]:
def lever_to_risk(portfolios, benchmark_index):
    benchmark = 'SPY'
    benchmark_index = standard[standard['benchmark']==benchmark]
    benchmark_risk = float(benchmark_index['stdev'])

In [445]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0


portfolios = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
portfolios.head()

,ret,stdev,sharpe,SPY,GLD,TLT,LQD
0,0.091165,0.099906,0.912506,0.202472,0.432783,0.032427,0.332317
1,0.074240,0.077016,0.963964,0.178256,0.002095,0.288567,0.531083
2,0.080532,0.081393,0.989420,0.071692,0.233153,0.264413,0.430742
3,0.083411,0.077150,1.081157,0.191168,0.209970,0.207740,0.391122
4,0.090172,0.083184,1.084012,0.242332,0.169393,0.502377,0.085898


In [473]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0


portfolios = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf).copy()
portfolios.head()


benchmark = 'SPY'
margin_int = 0.02
opt_levered_portfolio = match_risk(portfolios, benchmark, start, end, margin_int)
opt_levered_portfolio

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,ret,stdev,sharpe,SPY,GLD,TLT,LQD,levered ret,levered stdev,levered sharpe,leverage
1018,0.089639,0.079713,1.124518,0.295119,0.165204,0.382117,0.15756,0.194368,0.199594,0.973821,2.503904


In [ ]:
# clarify the math here

def match_return(portfolios, benchmark, start, end, margin_int):
    standard = standard = get_benchmark([benchmark],start, end)
    benchmark_index = standard[standard['benchmark']==benchmark]
    benchmark_return = float(benchmark_index['return'])
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    
    leverage = float(benchmark_return / optimal_portfolio['ret']) *
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

In [476]:
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104145,0.199594,0.521784
2,IWM,0.0968185,0.24876,0.389205
3,VT,0.0850438,0.22475,0.378392
4,QQQ,0.157312,0.214189,0.734452


In [ ]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0


portfolios = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf).copy()
portfolios.head()


benchmark = 'SPY'
margin_int = 0.02
opt_levered_portfolio = match_risk(portfolios, benchmark, start, end, margin_int)
opt_levered_portfolio